In [1]:
!pip install fastai==2.7.9
!pip install fastkaggle==0.0.7
!pip install fasterai==0.1.11
!pip install ipython-autotime==0.3.1
!pip install torch==1.12.1
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.11
    Uninstalling fastai-2.7.11:
      Successfully uninstalled fastai-2.7.11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
time: 569 µs (started: 2023-05-28 04:10:06 +00:00)


In [2]:
from fastai.vision.all import *
from fastai.callback.all import *
from fastai.data.all import *
from fastai.imports import *
from fastai.torch_core import *
from fastai.learner import *
from fastai.fp16_utils import *
from fastai.callback.fp16 import *
import fastai.optimizer as optim


from fasterai.sparse.all import *
from fasterai.core.criteria import *
from fasterai.regularize.all import *

from fastkaggle import *

import torch
import torch.nn as nn
import torch.nn.functional as F


set_seed(42)

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")


time: 3.74 s (started: 2023-05-28 04:10:06 +00:00)


In [3]:
comp = 'paddy-diseases/paddy-diseases'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

time: 16.3 s (started: 2023-05-28 04:10:10 +00:00)


In [4]:
dls = ImageDataLoaders.from_folder(path/"train", 
                                   valid_pct=0.25, 
                                   item_tfms=Resize(256), 
                                   batch_tfms=
                                   aug_transforms(   size=64, 
                                                                min_scale=0.75,
                                                                max_rotate=10., 
                                                                min_zoom=0.8, 
                                                                max_zoom=1.2, 
                                                            
                                                            ),
                                   bs=32)

time: 7.99 s (started: 2023-05-28 04:10:26 +00:00)


In [5]:
model = vgg16_bn().cuda()

time: 3.18 s (started: 2023-05-28 04:10:35 +00:00)


In [6]:
cbs = [     
#         GradientAccumulation(32), 
        SparsifyCallback(sparsity=12.5, 
                         granularity='filter', 
                         context='local', 
                         criteria=large_final, 
                         schedule=one_cycle),
        ]

time: 696 µs (started: 2023-05-28 04:10:38 +00:00)


In [7]:
from fastai.learner import Learner

class CustomLearner(Learner):
    optimizer_memory_usage = 0
    def _step(self):
        before_memory = torch.cuda.max_memory_allocated(device=torch.device("cuda"))
        super()._step()
        after_memory = torch.cuda.max_memory_allocated(device=torch.device("cuda"))
      
        # calculate the memory usage during the optimizer step
        self.optimizer_memory_usage += after_memory - before_memory
    
        print(f"Optimizer memory usage: {self.optimizer_memory_usage} Byte")


time: 1.2 ms (started: 2023-05-28 04:10:38 +00:00)


In [8]:
learn = CustomLearner(dls, 
                model = model, 
                metrics=[accuracy,
                         error_rate,
                        ],
               )
learn.unfreeze()
learn.recorder.train_metrics = True
learn.path = Path("/kaggle/working/")



time: 8.09 ms (started: 2023-05-28 04:10:38 +00:00)


In [9]:
learn.fit_one_cycle(1, 3e-5, cbs=cbs)

Pruning of filter until a sparsity of [12.5]%
Saving Weights at epoch 0


epoch,train_loss,train_accuracy,train_error_rate,valid_loss,valid_accuracy,valid_error_rate,time
0,2.439772,0.177724,0.822276,2.022501,0.307692,0.692308,01:33


Optimizer memory usage: 1407602176 Byte
Optimizer memory usage: 1479902208 Byte
Optimizer memory usage: 1479902208 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
Optimizer memory usage: 1480229888 Byte
